# US Census Data API/Data Collection
## ACS Survey 5

The ACS Survey 5 contains data from 2019 to 2022 and includes social, economic, demographic, and housing data all the way down to the zip code level. Our group has decided to use zip code level data for our analysis. We can then split zip codes into urban/rural and into various household income buckets. We can also summarize across the state.

In [1]:
# Dependencies
# Requests will let us send HTTP requests and return a result. 
# Numpy will let us perform various mathematical functions that may be requried
# Pandas lets us work with dataframes
# Census lets us submit simplified census API requests
# The Pypi library 'states' provides FIPS lookup functionality as well as other
# geography codes
# The config.py file contains the census API key
# And matplotlib will allow the creation of visualizations.

import requests
import numpy as np
import pandas as pd
from census import Census
from us import states
from config import census_api_key
import matplotlib.pyplot as plt

## Data Requirements

So, what data will we need to collect? 

For all zip codes in california: 
For the five years we are interested in:
-        What is the population by zip code?
-        What is the geographic area by zip code
-        What is the household income by zip code

It may be worth exploring to see if there are other data that would be helpful for answering some intriguing questions, but this should be enough to cover the questions already identified in our project. We can marry these data with the information on car registration by fuel from the california government website as well as California energy department info on energy production.

At present, I don't have geographic area by zip code. Census does not publish this. They do publish population density, but not by zip code. May need to summarize by county since zip codes are by residence, not technically a geographic area. See the "Alternate Data Sets" section for these results.

In [2]:
# Create an instance of the Census library
# Run Census Search to retrieve data on all zip codes (2018 to 2022 ACS5 Census--latest five years)
# We'll need a for loop to go through the years from 2018 to 2022
# For loops stop just before the last member of the range since it is a zero index system

multi_census_pd = pd.DataFrame()

for year in range(2018, 2023):
    c = Census(
        census_api_key,
        year = year
    )
    census_data = c.acs5.get(
        (
            "NAME",
            "B19013_001E",
            "B01003_001E",
        ),
        {'for': 'zip code tabulation area:*'}
    )

    # Convert to DataFrame
    census_pd = pd.DataFrame(census_data)

    # Column renaming
    census_pd = census_pd.rename(
        columns = {
            "NAME": "Name",
            "B19013_001E": "Household Income",
            "B01003_001E": "Population",
            "zip code tabulation area": "Zipcode"
        }
    )

    # Configure the final DataFrame (basically, just drop the "Name" column)
    census_pd = census_pd[
        [
            "Zipcode",
            "Population",
            "Household Income",
        ]
    ]
    census_pd['Year'] = year

    multi_census_pd = pd.concat([census_pd, multi_census_pd], axis=0)

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

## Filters/Slices

I will need to filter this dataset down to just show California zip codes and add in their city, longitude, latitude in case
we want to plot something on a map, or in case we want to group by city or something like that. 

In [ ]:
# In order to filter by state, we can use a csv file that is available from US census containing all 
# states, cities, zips, and the state_fips abbreviation.

zip_state_pd = pd.read_csv("Resources/geo-data.csv")

# Now, we just need to merge our two dataframes to get just the California values
multi_census_pd = multi_census_pd.merge(zip_state_pd, how='inner', left_on = 'Zipcode', right_on = 'zipcode', copy='false')

In [ ]:
# OK, now let's do some cleanup and drop states we don't need...
multi_census_pd.drop(columns='zipcode', inplace=True)
multi_census_pd = multi_census_pd[multi_census_pd['state_abbr'] == 'CA']

## Data Cleansing and Output

I will need to cleanse the data a bit so that I get rid of NaN's and negative income values in the raw census data.


In [ ]:
# We need to drop the following rows:
#     1) Any rows where the population is zero (to reduce risk of dividing by zero)
#     2) Any rows where Household Income is negative
#     3) Any rows missing cities (in case we end up agregating by city)
#     4) Any rows missing counties (in case we end up agregating by county)
#     5) Any rows where the city is just a ZCTA (zip code), not an actual city name
#     6) Any rows where the count by zip code is less than 5, so we don't have mismatched data by year
#        --basically situations where maybe there are data for one year for a given zip, but not for another year

multi_census_pd = multi_census_pd[multi_census_pd['Population']!= 0]
multi_census_pd = multi_census_pd[multi_census_pd['Household Income']> 0]
multi_census_pd = multi_census_pd[multi_census_pd['city'].notna()]
multi_census_pd = multi_census_pd[multi_census_pd['county'].notna()]
multi_census_pd = multi_census_pd[multi_census_pd['city'].str.startswith('Zcta') == False]

# To do number 6 above, we need to first make a list of zip codes
# Where the row count is less than five (since we ran this for five years)
keepzip = multi_census_pd.groupby('Zipcode').count()
keepzip = keepzip.loc[keepzip['city'] == 5]
# Now we can delete any rows from multi_census_pd
# Where the Zipcode is not in the 'keepzip' file
multi_census_pd = multi_census_pd[multi_census_pd['Zipcode'].isin(keepzip.index)]

multi_census_pd


In [ ]:
# Now we can write out our data file for incorporation into the master Pandas notebook
multi_census_pd.to_csv("Resources/census_data.csv", header=True, index=False)

## Alternate Data Set (by County rather than Zip since not all data available by Zip)
Not all of the diferent datasets are available by zip code, so I need to prepare a couple different files:
- Census data by county
- A cross reference from zip code to county

In [ ]:
# Create an instance of the Census library
# Run Census Search to retrieve data on all counties (2018 to 2022 ACS5 Census--latest five years)
# We'll need a for loop to go through the years from 2018 to 2022
# For: loops stop just before the last member of the range since it is a zero index system
# We'll start by creating an empty dataframe that we will hold all of our data for each year...

multi_census_county = pd.DataFrame()

for year in range(2018, 2023):
    c = Census(
        census_api_key,
        year = year
    )
    census_data = c.acs5.get(
        (
            "NAME",
            "B19013_001E",
            "B01003_001E"
        ),
        {'for': 'county:*', 'in': 'state:06'}
    )


    # Convert to DataFrame
    census_pd = pd.DataFrame(census_data)

    # Column renaming
    census_pd = census_pd.rename(
        columns = {
            "NAME": "Name",
            "B19013_001E": "Household Income",
            "B01003_001E": "Population",
            "county": "County FIPS"
        }
    )


    
    # Configure the final DataFrame (basically, just drop the "Name" column)
    census_pd = census_pd[
        [
            "County FIPS",
            "Population",
            "Household Income",
        ]
    ]
    census_pd['Year'] = year

    multi_census_county = pd.concat([census_pd, multi_census_county], axis=0)


In [ ]:
# But our county FIPS does not tell us the name of the county.
# We can pull in a CSV with a FIPS lookup value for California taken from US Census SF1

# We need to make sure the FIPS column isn't interpreted as a number, so we specify the dtype explicitly
county_name_pd = pd.read_csv('Resources/census_county_name_fips.csv', dtype={
    'Name': 'string',
    'FIPS': 'string'
})

In [ ]:
# So, now we will join our county name cross reference to our dataset...
multi_census_county = multi_census_county.merge(county_name_pd,
                                                how='inner',
                                                left_on = 'County FIPS',
                                                right_on = 'FIPS',
                                                copy = 'false'
                                               )
# Then, we will do a little cleanup--renaming columns, dropping the FIPS columns, and reordering...
multi_census_county = multi_census_county.rename(columns={"Name": "County"})
multi_census_county.drop('County FIPS', axis=1, inplace=True)
multi_census_county.drop('FIPS', axis=1, inplace=True)
multi_census_county = multi_census_county[['County', 'Year', 'Population', 'Household Income']]

In [ ]:
# While this dataset appears to be much cleaner than the ZIP dataset,
# just as a precaution we will run whatever cleanup code still applies
# from our analysis of the ZIP dataset.

multi_census_county = multi_census_county[multi_census_county['Population']!= 0]
multi_census_county = multi_census_county[multi_census_county['Household Income']> 0]
multi_census_county = multi_census_county[multi_census_county['County'].notna()]

multi_census_county

In [ ]:
# Now we can write our data set to a CSV file for any of the collaborators to use...
multi_census_county.to_csv("Resources/census_data_by_county.csv", header=True, index=False)

In [ ]:
# We still need to make a file containing the zip codes and the counties
# so that we can summarize by county in situations where there is no zipcode level information

zipcode_county_pd = multi_census_pd.loc[multi_census_pd['Year'] == 2022]
zipcode_county_pd = zipcode_county_pd[
        [
            "Zipcode",
            "county"
        ]
    ]

In [ ]:
# And now we will write out our zip/county cross reference file for anyone who needs it...
zipcode_county_pd.to_csv("Resources/zip_to_county.csv", header=True, index=False)

## Extrapolation
Since not all our datasets are available for the same years, as a team we decided
that we would extrapolate out the census data to the year 2023 in order to come up with the most
recent five year period for analysis. 

In [ ]:
# Pseudocode: for the multi_census_county dataframe, we will want to group by the county, grab the 2018 value for each of population and
# household income, grab the 2022 value, find the difference, divide by four (the number of years of change). This will
# be the average yearly change. 
#
# Then we will create a 2023 value of each of population and household income that is the sum of the 2022 value and this delta.

# Using Alameda County as an example, grab the 2022 value of "Population" (1663823), the 2018 value (1643700), find the difference (20123),
# divide by 4, convert to an integer since you can't have fractional people (5031) , And add that value to the 2022 value so we get 
# an extrampolated 2023 population value for Alameda County of 1668854- a slight increase over 2022.
#
# The same can be done for Household income. It has gone (in Alameda) from 92574 in 2018 to 122488 in 2022. That's an increae of 29914 in 4 years.
# So that's an average increase of 7479 per year (integer value again). So we add 7479 to our 2022 number and get 129967 as our 2023 household income.


delta_start = multi_census_county.loc[multi_census_county['Year'] == 2018]
delta_start = delta_start.rename(columns={"Population": "Start Pop",
                                          "Household Income": "Start Income"
                                         })
delta_end = multi_census_county.loc[multi_census_county['Year'] == 2022]
delta_end = delta_end.rename(columns={"Population": "End Pop",
                                          "Household Income": "End Income"
                                       })
delta_start.drop('Year', axis=1, inplace=True)
delta_end.drop('Year', axis=1, inplace=True)
delta = delta_start.merge(delta_end, how='inner', on='County')
delta["Pop Delta"] = round((delta["End Pop"] - delta["Start Pop"]) / 4, 0)
delta["Inc Delta"] = round((delta["End Income"] - delta["Start Income"]) / 4, 0)
delta["Population"] = delta["End Pop"] + delta["Pop Delta"]
delta["Household Income"] = delta["End Income"] + delta["Inc Delta"]
delta.drop(['Start Pop', 'Start Income', 'End Pop', 'End Income', 'Pop Delta', 'Inc Delta'], axis=1, inplace=True)
delta["Year"] = 2023
delta = delta[['County', 'Year', 'Population', 'Household Income']]
census_county_extrapolate = pd.concat([delta, multi_census_county], axis=0)
census_county_extrapolate = census_county_extrapolate.sort_values(by=['County','Year'],ignore_index=True)

## Include Geograpic Area by County

In order to determine whether there are differences in adoption by county, we will need to know the area of each county, then we can calculate population density by dividing population by area.

In [ ]:
# First, we import a data file containing the area in square miles for each county

county_area_pd = pd.read_csv("Resources/census_county_area.csv")

# Now, we can merge this with our extrapolated, complete dataset...

census_county_extrapolate = census_county_extrapolate.merge(county_area_pd, how='inner', on='County')
census_county_extrapolate["Density"] = census_county_extrapolate["Population"] / census_county_extrapolate["Area (sq mi)"]

# Time to write out our results...

census_county_extrapolate.to_csv("Resources/census_data_by_county_with_2023.csv", header=True, index=False)

In [ ]:
census_county_extrapolate

## Merge Datasets for Visualizations

Now, we need to bring in data created by other collaborators.  First, we will bring in the DMV data generated by Gina.

In [ ]:
# First, we need to import the DMV data into a Pandas dataframe...

battery_electric_df = pd.read_csv('battery_electric_totals.csv')

In [ ]:
battery_electric_df.head()

In [ ]:
# Now we can merge those data with the existing census dataframe so that we have
# the combination of County, Year, Population, Income, Pop Density, and number of
# vehicles registered that year

merged_census_DMV = census_county_extrapolate.merge(battery_electric_df,
                                                    how='inner',
                                                    left_on = ['County', 'Year'],
                                                    right_on = ['county', 'Year'],
                                                    copy='false')

# We can drop a couple column that aren't needed. First, we have two county columns, so dropping one
# Second, we don't need to specify the fuel since the only records in the dataframe are EV's
merged_census_DMV.drop('county', axis=1, inplace=True)
merged_census_DMV.drop('Fuel', axis=1, inplace=True)

merged_census_DMV.loc[merged_census_DMV['County'] == 'Los Angeles']

In [ ]:
# For purposes of determining Urban vs Rural counties, the US Census bureau uses a combination of factors
# involving both the size of the population center (number of total people in the city) as well as 
# the population density. However, this does not perfectly match with either zip code or county. From examining
# the California data, the best approximation is that all (or nearly all) counties with a population density 
# greater than 1,000 people per square mile is a good cutoff point.  So, we will classify our counties as either urban
# or rural (those are the only two groupings for the US Census) based upon this threshold.
#
# In order to minimize the risk of unintended results in the visualizations, I will use the 2022 numbers
# of population density to assign a value of "urban" or "rural" to each county, regardless of whether
# other years are slightly above or below the threshold. 
# 
# To do this, I will calculate the value for all rows, then overwrite years other than 2022 with the 2022 
# classification

# First, set a starting value of Urban...
# then update it to Rural if the density is below 1,000 people per square mile
merged_census_DMV['Classification'] = 'Urban'
merged_census_DMV['Classification'] = merged_census_DMV['Classification'].where(merged_census_DMV['Density'] > 1000, 'Rural')

# Now, we need to make sure any given county isn't fluctuating from one year to the next...
# So, we will make a dataframe with just the 2022 values
merged_census_DMV_2022 = merged_census_DMV.loc[merged_census_DMV['Year'] == 2022]
# Then, we will set to the classification for all years based on the 2022 values
# By merging the two dataframes together
# Then dropping the 'Classification' in favor of the '2022 Classification'
merged_census_DMV = merged_census_DMV.merge(merged_census_DMV_2022,
                                            how='inner',
                                            on='County',
                                            copy = 'false'
                                           )
# Now we just need to clean up the result...
merged_census_DMV.drop(['Classification_x',
                      'Year_y',
                      'Population_y',
                      'Household Income_y',
                      'Area (sq mi)_y',
                      'Density_y',
                      'Vehicles_y',
                      'Classification_x'], 
                      axis=1, 
                      inplace=True)
merged_census_DMV.columns = ['County',
                             'Year',
                             'Population',
                             'Household Income',
                             'Area (sq mi)',
                             'Density',
                             'Vehicles',
                             'Classification'
                            ]
merged_census_DMV


In [ ]:
# Let's write out that dataframe in case anyone else needs it for their visualizations...
merged_census_DMV.to_csv("Resources/census_and_dmv_data.csv", header=True, index=False)

## Aggregations
We will need to aggregate our data set different ways in order to support different visualizations. The first question from our charter is, "What is the adoption rate for EV's in urban vs rural areas?" To answer that, we need to aggregate our data so that instead of each county, we are looking at two data sets, by year, one for rural and one for urban.

In [ ]:
# The first question from our charter is, "What is the adoption rate for
# EV's in urban vs. rural areas?"  To answer that, we need to aggregate our databy Classification
# and we will want a summary data set for the entire state. That way we can plot two separate
# data sets in the same visualization

rural_ev_adoption = merged_census_DMV.loc[merged_census_DMV['Classification'] == 'Rural']
urban_ev_adoption = merged_census_DMV.loc[merged_census_DMV['Classification'] == 'Urban']
summary_rural_adoption = rural_ev_adoption.groupby('Year').agg({'Vehicles': 'sum'})
summary_urban_adoption = urban_ev_adoption.groupby('Year').agg({'Vehicles': 'sum'})
aggregated_rural = rural_ev_adoption.groupby('Year').agg({'Vehicles': 'sum', 'Population': 'sum'})
aggregated_urban = urban_ev_adoption.groupby('Year').agg({'Vehicles': 'sum', 'Population': 'sum'})
aggregated_rural['Vehicles per 100K'] = aggregated_rural['Vehicles'] / (aggregated_rural['Population'] / 100000)
aggregated_urban['Vehicles per 100K'] = aggregated_urban['Vehicles'] / (aggregated_urban['Population'] / 100000)

# To give a little more granularity, we'll want to run this by county as well. The chart
# is likely to get a bit "busy", though, if we include all 58 counties (or the 56 we have valid data
# for), so let's see how many urban and rural counties we need to include (same number of each)
# in order to capture 75% of the California population... We can use 2022 data to see

total_population = merged_census_DMV.groupby('Year').sum()
total_population = total_population.loc[total_population.index == 2022]['Population']
total_population

In [ ]:
# So the total population in California, per the 2022 census, is 39,000,000 people
# Let's see how many counties we need, both urban and rural, to capture 29,000,000 people. 
# Using this subset will make our visualization much less cluttered and will still be
# a good representation of the state as a whole.
# Trial and error will probably be easiest. Let's see what percentage of the population we get if we
# choose the six largest urban counties and six largest rural counties...

rural_ev_adoption_2022 = rural_ev_adoption.loc[rural_ev_adoption['Year'] == 2022]
rural_ev_top8 = rural_ev_adoption_2022.sort_values(by='Population',ignore_index=True, ascending=False)
rural_ev_top8 = rural_ev_top8.iloc[:8]
rural_ev_top8['Population'].sum()


In [ ]:
# So, we have 12 million people by choosing the population of the 6 largest rural counties. Let's do
# the same for the ten largest urban counties...

urban_ev_adoption_2022 = urban_ev_adoption.loc[urban_ev_adoption['Year'] == 2022]
urban_ev_top8 = urban_ev_adoption_2022.sort_values(by='Population',ignore_index=True, ascending=False)
urban_ev_top8 = urban_ev_top8.iloc[:8]
urban_ev_top8['Population'].sum()

In [ ]:
# So, that gets us about 33 million people or 85% of the population. That seems like a 
# reasonable compromise between creating a visualization that is overly busy, and a visualization
# that doesn't have a good representative set of counties.
# 
# We'll need the dataset across all three years, though, so let's grab all the rural
# and all the urban data, but just using the counties in our "top7" list...

rural_ev_adoption_largest = rural_ev_adoption[rural_ev_adoption['County'].isin(rural_ev_top8['County'].tolist())].reindex()
urban_ev_adoption_largest = urban_ev_adoption[urban_ev_adoption['County'].isin(urban_ev_top8['County'].tolist())].reindex()

# We need to normalize these data by looking at vehicles per 100,000 population... Same as we did for the full
# state data earlier.

rural_ev_adoption_largest['Vehicles per 100K'] = rural_ev_adoption_largest['Vehicles'] / (rural_ev_adoption_largest['Population'] / 100000)
urban_ev_adoption_largest['Vehicles per 100K'] = urban_ev_adoption_largest['Vehicles'] / (urban_ev_adoption_largest['Population'] / 100000)
rural_ev_adoption_largest.drop(['Population',
                                'Household Income',
                                'Area (sq mi)',
                                'Density',
                                'Classification',
                                'Vehicles'],
                               axis=1,
                               inplace=True)
urban_ev_adoption_largest.drop(['Population',
                                'Household Income',
                                'Area (sq mi)',
                                'Density',
                                'Classification',
                                'Vehicles'],
                               axis=1,
                               inplace=True)
urban_ev_adoption_largest

# That should give us the exact data we need for a stacked bar chart visualization

## Summary of Results
Below, the final results for the state as a whole are calculated as well as for the eight largest rural and eight largest urban counties.

In [ ]:
print(f'Average annual urban registrations: {aggregated_urban["Vehicles per 100K"].mean()}')
print(f'Average annual rural registrations: {aggregated_rural["Vehicles per 100K"].mean()}')

## Visualizations
We should now have all the datasets required to answer our questions. In order to facilitate analysis, it is likely easiest to create visualizations. This will show trends, by county and also at the summary level by state.

In [ ]:
# Let's try plotting the adoption rate for the entire state of EV's
# as a line chart, separating out the number of car registrations into rural vs. urban areas
#
# Rather than just looking at the totals, a more interesting comparison is to look at 
# The number of cars registered per hundred thousand people, otherwise we will just see
# higher numbers for urban regions because they are the majority of the population, when what
# we really want are the comparative rates of adoption, so the 'y' axis is effectively
# normalized for both urban and rural--a true rate.

ax = aggregated_urban['Vehicles per 100K'].plot(xticks=aggregated_rural.index,
                                    title='Total California EV Car Registrations, Urban and Rural',
                                   )
aggregated_rural['Vehicles per 100K'].plot(ax = ax, xticks=aggregated_rural.index)
plt.legend(["Urban EVs Registered", "Rural EVs Registered"])
plt.ylabel('Cars Registered per Year per 100K Population')
title='California EV Car Registrations, Urban and Rural'

plt.savefig('Resources/Car_Registrations_Urban_v_Rural.png')

plt.show()

In [ ]:
# Let's try plotting the same data, but by county rather than at the summary level
# for the entire state. This will probably work best as a stacked bar chart
# with the cars per year per 100K population on the 'y' axis, and the individual
# counties along the x axis. To keep it from becoming too busy, maybe ten largest
# (by population) urban counties vs. ten largest rural counties? That would cover
# 
pivot_rural = pd.pivot_table(data=rural_ev_adoption_largest,
                             index=['County'],
                             columns=['Year'],
                             values='Vehicles per 100K')
pivot_urban = pd.pivot_table(data=urban_ev_adoption_largest,
                             index=['County'],
                             columns=['Year'],
                             values='Vehicles per 100K')
pivot_rural = pivot_rural.sort_values(by=2022, ignore_index=False, ascending=True)
pivot_urban = pivot_urban.sort_values(by=2022, ignore_index=False, ascending=True)

# We are going to want two subplots, right next to each other
# Any other presentation gets muddled since there is no way to tell
# by county name, for example, whether it is urban or rural


fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
fig.set_figwidth(12)
fig.set_figheight(6)
fig.suptitle('EV Adoption by year for Seven Largest Rural and Urban Counties')
ax1y1 = pivot_rural[2020].tolist()
ax1y2 = pivot_rural[2021].tolist()
ax1y3 = pivot_rural[2022].tolist()
ax2y1 = pivot_urban[2020].tolist()
ax2y2 = pivot_urban[2021].tolist()
ax2y3 = pivot_urban[2021].tolist()
ax1.set_ylabel('Vehicles Registered per 100K')


ax1.bar(pivot_rural.index.tolist(), ax1y1, color='r', label='2020')
ax1.bar(pivot_rural.index.tolist(), ax1y2, bottom = ax1y1, color='b', label='2021')
ax1.bar(pivot_rural.index.tolist(), ax1y3, bottom = np.add(ax1y1, ax1y2), color='g', label='2022')
ax1.legend(loc="upper left")
ax2.bar(pivot_urban.index.tolist(), ax2y1, color='r', label='2020')
ax2.bar(pivot_urban.index.tolist(), ax2y2, bottom = ax2y1, color='b', label='2021')
ax2.bar(pivot_urban.index.tolist(), ax2y3, bottom = np.add(ax2y1, ax2y2), color='g', label='2022')
ax2.legend(loc="upper left")

ax1.tick_params(axis='x', labelrotation=90)
ax2.tick_params(axis='x', labelrotation=90)
ax1.set_xlabel('Largest Rural Counties')
ax2.set_xlabel('Largest Urban Counties')
fig.align_xlabels()

plt.tight_layout()
plt.savefig('Resources/Car_Registrations_Urban_v_Rural_by_County.png')

plt.show()


# Analysis

Going into the project, I was expecting to see a large gap in the adoption rate of EV's between urban and rural counties. The expectation was that the longer distances required to get to a public charging station coupled with the longer driving distances in general for residents of rural counties would significantly reduce the number of EV's purchased and operated in rural counties on a per capita basis. That turned out not to be the case. Rural counties have an average of 218 EV car registrations per year per 100,000 residents across the three years for which DMV data were available, while urban counties had an average of about 234 EV car registrations per year per 100,000, only 7% higher than rural counties. In addition, that gap is narrowing with 2022 data almost exactly matching between urban and rural locations in California.

In looking at the data for the eight largest counties, both urban and rural, there is one trend to note: adoption rates appear somewhat higher in total over the three years for northern california urban counties than for southern california urban counties. The differences in the rural counties are much smaller with adoption rates being vary similar across the board. 

There are two significant weaknesses in the datasets that should be pointed out. First, only three years of DMV were available, so extrapolation of current trends into future years would be quite risky. Second, the classification of rural vs. urban may be generally correct, but there are some counties that have significant urban populations--perhaps even mostly urban populations--that are considered "rural" simply because they are so large. The most obvious of these is San Diego county. 

# Ending of Jared (census_jared.ipynb)

# Beginning of Gina (Ginaupdates.ipynb)

In [ ]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
import scipy.stats as st
import matplotlib.pyplot as plt
import numpy as np

# yearly files
vehicle_data_2020 = ('vehicle_type_by_zipcode_2020.csv')
vehicle_data_2021 = ('vehicle_type_by_zipcode_2021.csv')
vehicle_data_2022 = ('vehicle_type_by_zipcode_2022.csv')

#census data
census_data = ('census_data.csv')

# reading each file
data_2020 = pd.read_csv(vehicle_data_2020, low_memory=False)
data_2021 = pd.read_csv(vehicle_data_2021, low_memory=False)
data_2022 = pd.read_csv(vehicle_data_2022, low_memory=False)
county_data = pd.read_csv(census_data)

# adding a year column
data_2020['Year'] = 2020
data_2021['Year'] = 2021
data_2022['Year'] = 2022

# Concatenate the DataFrames
merged_data = pd.concat([data_2020, data_2021, data_2022], ignore_index=True)

# resetting the index of the DataFrame
merged_data.reset_index(drop=True, inplace=True)

# Display the merged DataFrame
print(merged_data)


merged_data.head()

In [ ]:
records_count = len(merged_data)
records_count

In [ ]:
fuel_types = merged_data['Fuel'].unique()

# Print the unique fuel types in this set
print("Fuel types:", fuel_types)

In [ ]:
# Dropping the columns for model year, make, and duty
merged_data.drop(['Model Year', 'Make', 'Duty'], axis=1, inplace=True)
merged_data.rename(columns={'Zip Code': 'Zipcode'}, inplace=True)
merged_data.head()

In [ ]:
# saving the merged data to a new csv, after the dropped fields
merged_data.to_csv('merged_data_modified.csv', index=False)

In [ ]:
# merging the vehicle data to the zip and county data 
census_data = ('census_data.csv')
vehicle_data_updt = ('merged_data_modified.csv')

# reading each file
county_data = pd.read_csv(census_data, dtype={
    'Zipcode': 'object',
    'county': 'object'})
all_vehicle_data = pd.read_csv(vehicle_data_updt, low_memory=False)

In [ ]:
county_data.drop(['Unnamed: 0', 'Population', 'Household Income', 'state_fips', 'state', 'state_abbr', 'city', 'Year'], axis=1, inplace=True)
county_data.head()

In [ ]:
# Merge the dataframes on the 'Zip Code' column
merged_data_all = pd.merge(all_vehicle_data, county_data, on='Zipcode', how='left')

# Display the merged dataframe
merged_data_all.head()
merged_data_all

In [ ]:
print(merged_data_all['Year'].unique())

In [ ]:
desired_county = 'Alameda'
desired_year = 2022

# filtering dataframe for the desired county and year
county_year_data = merged_data_all.loc[(merged_data_all['county'] == desired_county) & (merged_data_all['Year'] == desired_year)]
county_year_data

In [ ]:
# getting all the data for 2020 with counties
fuel_types_2020 = merged_data_all[merged_data_all['Year'] == 2020]
fuel_types_2020 = fuel_types_2020.groupby(['county','Fuel', 'Year']).agg({'Vehicles': 'count'}).reset_index()
fuel_types_2020

In [ ]:
total_vehicle_counts_2020 = fuel_types_2020.groupby('Fuel')['Vehicles'].sum().reset_index()

total_vehicle_counts_2020

In [ ]:
# getting all the data for 2020 with counties
fuel_types_2021 = merged_data_all[merged_data_all['Year'] == 2021]
fuel_types_2021 = fuel_types_2021.groupby(['county','Fuel', 'Year']).agg({'Vehicles': 'count'}).reset_index()
fuel_types_2021

In [ ]:
total_vehicle_counts_2021 = fuel_types_2021.groupby('Fuel')['Vehicles'].sum().reset_index()

total_vehicle_counts_2021

In [ ]:
fuel_types_2022 = merged_data_all[merged_data_all['Year'] == 2022]
fuel_types_2022 = fuel_types_2022.groupby(['county','Fuel', 'Year']).agg({'Vehicles': 'count'}).reset_index()
fuel_types_2022

In [ ]:
total_vehicle_counts_2022 = fuel_types_2022.groupby('Fuel')['Vehicles'].sum().reset_index()

total_vehicle_counts_2022

In [ ]:
# Data for 2020
fuel_types_2020 = {
    'Battery Electric': 76020,
    'Diesel and Diesel Hybrid': 295075,
    'Flex-Fuel': 259780,
    'Gasoline': 1586575,
    'Hybrid Gasoline': 201905,
    'Hydrogen Fuel Cell': 8415,
    'Natural Gas': 26260,
    'Other': 7770,
    'Plug-in Hybrid': 79645
}

# Data for 2021
fuel_types_2021 = {
    'Battery Electric': 85820,
    'Diesel and Diesel Hybrid': 331395,
    'Flex-Fuel': 339580,
    'Gasoline': 1713775,
    'Hybrid Gasoline': 211950,
    'Hydrogen Fuel Cell': 10490,
    'Natural Gas': 16750,
    'Other': 47445,
    'Plug-in Hybrid': 96090
}

# Data for 2022
fuel_types_2022 = {
    'Battery Electric': 106290,
    'Diesel and Diesel Hybrid': 367745,
    'Flex-Fuel': 350785,
    'Gasoline': 1834675,
    'Hybrid Gasoline': 235670,
    'Hydrogen Fuel Cell': 13705,
    'Natural Gas': 29005,
    'Other': 6280,
    'Plug-in Hybrid': 110320
}

# Extract fuel types and vehicle totals
fuel_types = list(fuel_types_2020.keys())
vehicles_2020 = list(fuel_types_2020.values())
vehicles_2021 = list(fuel_types_2021.values())
vehicles_2022 = list(fuel_types_2022.values())

# Plotting
plt.figure(figsize=(9, 5))
plt.plot(fuel_types, vehicles_2020, label='2020')
plt.plot(fuel_types, vehicles_2021, label='2021')
plt.plot(fuel_types, vehicles_2022, label='2022')

# Customizing the plot
plt.xlabel('Fuel Type')
plt.ylabel('Vehicle Total')
plt.title('Vehicle Total by Fuel Type (2020-2022)')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.grid(True)

# Show plot
plt.tight_layout()
plt.show()

In [ ]:
battery_electric_data = merged_data_all[merged_data_all['Fuel'] == 'Battery Electric']
#battery_electric_data = battery_electric_data.drop_duplicates()
battery_electric_data

In [ ]:
# filtering the DataFrame by 'Zip Code' to calculate the sum of 'Vehicles' for each zip code
battery_electric_totals = battery_electric_data.groupby(['county','Fuel', 'Year']).agg({'Vehicles': 'count'}).reset_index()

battery_electric_totals


In [ ]:
# dropping two counties: Sierra and Modoc, total counties will be 56

# filtering the DataFrame by ‘Zip Code’ to calculate the sum of ‘Vehicles’ for each zip code
battery_electric_totals = battery_electric_data.groupby(['county','Fuel', 'Year']).agg({'Vehicles': 'count'}).reset_index()
# We also need to drop any counties where we don’t have all three years of data...
# To do that, we want to find a list of counties
keepcounty = battery_electric_totals.groupby('county').count()
keepcounty = keepcounty.loc[keepcounty['Fuel'] == 3]
# Now we can delete any rows from multi_census_pd
# Where the Zipcode is not in the ‘keepzip’ file
battery_electric_totals = battery_electric_totals[battery_electric_totals['county'].isin(keepcounty.index)]
# totals for each zip code
battery_electric_totals

In [ ]:
# saving the merged data to a new csv, after the dropped fields
battery_electric_totals.to_csv('battery_electric_totals.csv', index=False)

In [ ]:
num_counties = battery_electric_totals['county'].nunique()
num_counties

BAR CHART FOR NUMBER OF EV PER COUNTY AND YEAR

In [ ]:
pivot_df = battery_electric_totals.pivot(index='county', columns='Year', values='Vehicles')

# Plotting
plt.figure(figsize=(12, 6))

# Define x positions for bars
x = np.arange(len(pivot_df.index))

# Width of each bar
width = 0.25

# Plot bars for each year
plt.bar(x - width, pivot_df[2020], width, label='2020')
plt.bar(x, pivot_df[2021], width, label='2021')
plt.bar(x + width, pivot_df[2022], width, label='2022')

# Add labels and title
plt.xlabel('County')
plt.ylabel('# of Electric Vehicles')
plt.title('Number of Electric Vehicles per County and Year')
plt.xticks(x, pivot_df.index, rotation=90)
plt.legend()

# Show plot
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

# Calculate the percentage increase from 2020 to 2021
increase_2021 = ((pivot_df[2021] - pivot_df[2020]) / pivot_df[2020]) * 100

# Calculate the percentage increase from 2021 to 2022
increase_2022 = ((pivot_df[2022] - pivot_df[2021]) / pivot_df[2021]) * 100

# Combine the increase rates into a DataFrame
increase_rates = pd.DataFrame({'Increase Rate 2021': increase_2021, 'Increase Rate 2022': increase_2022})

# Display the increase rates
print("Increase Rates:")
increase_rates


In [ ]:
total_by_year = battery_electric_totals.groupby('Year')['Vehicles'].sum()

# Display the total number of battery electric vehicles for each year
total_by_year

# Ending of Gina

Going into the project, I was expecting to see a large gap in the adoption rate of EV's between urban and rural counties. The expectation was that the longer distances required to get to a public charging station coupled with the longer driving distances in general for residents of rural counties would significantly reduce the number of EV's purchased and operated in rural counties on a per capita basis. That turned out not to be the case. Rural counties have an average of 218 EV car registrations per year per 100,000 residents across the three years for which DMV data were available, while urban counties had an average of about 2Going into the project, I was expecting to see a large gap in the adoption rate of EV's between urban and rural counties. The expectation was that the longer distances required to get to a public charging station coupled with the longer driving distances in general for residents of rural counties would significantly reduce the number of EV's purchased and operated in rural counties on a per capita basis. That turned out not to be the case. Rural counties have an average of 218 EV car registrations per year per 100,000 residents across the three years for which DMV data were available, while urban counties had an average of about 234 EV car registrations per year per 100,000, only 7% higher than rural counties. In addition, that gap is narrowing with 2022 data almost exactly matching between urban and rural locations in California.34 EV car registrations per year per 100,000, only 7% higher than rural counties. In addition, that gap is narrowing with 2022 data almost exactly matching between urban and rural locations in California.

In looking at the data for the eight largest counties, both urban and rural, there is one trend to note: adoption rates appear somewhat higher in total over the three years for northern california urban counties than for southern california urban counties. The differences in the rural counties are much smaller with adoption rates being vary similar across the board. 

There are two significant weaknesses in the datasets that should be pointed out. First, only three years of DMV were available, so extrapolation of current trends into future years would be quite risky. Second, the classification of rural vs. urban may be generally correct, but there are some counties that have significant urban populations--perhaps even mostly urban populations--that are considered "rural" simply because they are so large. The most obvious of these is San Diego county. 

# Beginning of Aram (EV_Chargers.ipynb)

In [ ]:
# Dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

#  Data
EV_Chargers_Path = "EV_Chargers1.csv"

# Read the data
EV_Chargers_data = pd.read_csv(EV_Chargers_Path)

# Preview data
EV_Chargers_data.head()

In [ ]:
# Data in the form of a list of lists
data = [
    ['Alameda', 5893, 4624, 4504, 4430, 4282, 5139, 5051, 3697, 3625, 3560, 3544, 3353, 3082, 2990],
    ['Alpine', 12, 10, 10, 6, 16, 16, 18, 14, 15, 15, 15, 15, 13, 12],
    ['Amador', 34, 34, 34, 34, 33, 33, 33, 39, 40, 40, 40, 40, 30, 29],
    ['Butte', 157, 149, 147, 146, 139, 158, 132, 94, 92, 91, 82, 76, 76, 74],
    ['Calaveras', 23, 23, 21, 25, 23, 23, 21, 21, 21, 19, 19, 7, 7, 7],
    ['Colusa', 37, 36, 36, 34, 34, 36, 36, 38, 34, 30, 28, 28, 28, 28],
    ['Contra Costa', 1632, 1734, 1674, 1503, 1495, 1699, 1715, 1451, 1483, 1461, 1455, 1355, 1325, 1068],
    ['Del Norte', 46, 47, 47, 39, 39, 32, 32, 29, 29, 25, 25, 30, 30, 30],
    ['El Dorado', 258, 214, 202, 189, 194, 193, 173, 179, 181, 177, 167, 173, 160, 150],
    ['Fresno', 1292, 1313, 1373, 1291, 1158, 1321, 1311, 1133, 1143, 1127, 1092, 921, 779, 684],
    ['Glenn', 8, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14],
    ['Humboldt', 157, 157, 157, 154, 150, 148, 145, 150, 142, 134, 124, 112, 114, 105],
    ['Imperial', 52, 50, 50, 49, 45, 43, 35, 31, 33, 33, 33, 27, 27, 15],
    ['Inyo', 83, 54, 52, 48, 49, 49, 41, 42, 39, 39, 35, 37, 23, 23],
    ['Kern', 832, 797, 721, 691, 646, 629, 631, 416, 359, 306, 298, 279, 279, 241],
    ['Kings', 431, 419, 410, 410, 176, 168, 160, 204, 148, 147, 147, 144, 140, 122],
    ['Lake', 18, 17, 14, 14, 15, 15, 14, 14, 17, 17, 15, 15, 15, 10],
    ['Lassen', 16, 16, 16, 16, 16, 16, 13, 13, 14, 6, 6, 5, 5, 4],
    ['Los Angeles', 31569, 29433, 29051, 27015, 25685, 24615, 23903, 20950, 20520, 20193, 19872, 19236, 18509, 16770],
    ['Madera', 203, 202, 203, 190, 161, 152, 147, 130, 122, 121, 121, 113, 89, 84],
    ['Marin', 1020, 1082, 1048, 1027, 933, 1052, 1037, 739, 734, 730, 728, 706, 684, 629],
    ['Mariposa', 62, 59, 60, 46, 33, 45, 35, 24, 30, 16, 16, 16, 16, 16],
    ['Mendocino', 229, 214, 203, 186, 200, 162, 158, 160, 158, 158, 146, 144, 129, 110],
    ['Merced', 182, 155, 124, 135, 177, 134, 139, 125, 142, 142, 138, 130, 114, 94],
    ['Modoc', 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 4, 4, 4, 4],
    ['Mono', 52, 38, 38, 38, 38, 37, 37, 32, 32, 32, 32, 32, 31, 31],
    ['Monterey', 491, 480, 487, 424, 394, 400, 383, 452, 425, 431, 424, 422, 360, 348],
    ['Napa', 478, 442, 433, 396, 391, 403, 393, 393, 390, 388, 386, 374, 348, 316],
    ['Nevada', 149, 144, 144, 169, 161, 152, 152, 116, 89, 88, 88, 93, 88, 86],
    ['Orange', 6771, 6002, 5797, 5383, 5214, 5189, 5003, 5852, 5657, 5477, 5381, 5242, 4886, 4799],
    ['Placer', 622, 522, 518, 455, 425, 448, 424, 419, 402, 365, 361, 351, 339, 305],
    ['Plumas', 7, 7, 7, 7, 7, 8, 7, 7, 7, 7, 7, 7, 5, 5],
    ['Riverside', 1835, 1835, 1745, 1635, 1508, 1493, 1441, 1548, 1495, 1441, 1422, 1464, 1360, 1306],
    ['Sacramento', 2253, 2205, 2149, 2040, 1756, 1839, 1765, 1813, 1836, 1852, 1850, 1777, 1765, 1438],
    ['San Benito', 27, 29, 28, 45, 43, 47, 47, 47, 49, 47, 47, 50, 42, 44],
    ['San Bernardino', 1957, 1841, 1767, 1682, 1615, 1560, 1424, 1367, 1478, 1415, 1395, 1338, 1297, 999],
    ['San Diego', 8583, 8094, 7960, 7735, 7358, 7074, 6551, 10080, 7278, 7199, 7170, 6874, 6760, 6073],
    ['San Francisco', 2431, 2070, 2002, 1952, 1436, 1575, 1519, 1630, 1840, 1671, 1609, 1505, 1384, 1264],
    ['San Joaquin', 825, 728, 680, 632, 612, 596, 565, 442, 407, 402, 397, 389, 326, 306],
    ['San Luis Obispo', 777, 705, 694, 650, 616, 640, 638, 566, 513, 513, 493, 493, 486, 442],
    ['San Mateo', 5533, 4798, 4619, 4557, 4436, 4799, 4688, 4511, 4701, 4459, 4282, 4070, 3758, 3617],
    ['Santa Barbara', 648, 526, 566, 512, 504, 515, 480, 557, 522, 492, 499, 494, 461, 437],
    ['Santa Clara', 20220, 18193, 17865, 17621, 16227, 16981, 16831, 15987, 16498, 16202, 16119, 15406, 14913, 14164],
    ['Santa Cruz', 359, 369, 356, 359, 345, 356, 333, 307, 306, 300, 302, 281, 262, 250],
    ['Shasta', 227, 179, 174, 153, 138, 105, 90, 86, 92, 93, 93, 96, 94, 57],
    ['Sierra', 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    ['Siskiyou', 77, 69, 68, 67, 67, 68, 68, 72, 68, 67, 56, 63, 63, 55],
    ['Solano', 608, 578, 560, 482, 473, 509, 501, 471, 421, 421, 394, 372, 362, 369],
    ['Sonoma', 902, 871, 845, 816, 794, 840, 765, 794, 803, 791, 789, 743, 695, 670],
    ['Stanislaus', 249, 202, 178, 188, 171, 158, 149, 151, 158, 150, 151, 145, 145, 122],
    ['Sutter', 35, 31, 24, 22, 27, 24, 24, 27, 33, 31, 31, 31, 23, 20],
    ['Tehama', 94, 91, 91, 87, 91, 80, 66, 35, 31, 30, 31, 32, 32, 32],
    ['Trinity', 15, 12, 14, 14, 14, 14, 14, 6, 6, 6, 6, 8, 8, 4],
    ['Tulare', 291, 275, 279, 231, 212, 160, 154, 123, 124, 119, 108, 103, 85, 76],
    ['Tuolumne', 55, 54, 54, 58, 60, 52, 52, 78, 54, 50, 50, 47, 47, 45],
    ['Ventura', 930, 875, 870, 858, 810, 821, 273, 14, 11, 11, 11, 11, 13, 749],
    ['Yolo', 468, 432, 427, 412, 398, 428, 679, 894, 894, 893, 892, 845, 868, 264],
    ['Yuba', 69, 80, 79, 75, 59, 60, 299, 394, 388, 386, 386, 324, 328, 14],
    ['Unknown', 3368, 807, 807, 807, 1868, 16, 41, 32, 16, 16, 16, 16, 16, 16]
]

# Columns for the DataFrame
columns = ['County', 'Q4 2023', 'Q3 2023', 'Q2 2023', 'Q1 2023', 'Q4 2022', 'Q3 2022', 'Q2 2022', 
           'Q1 2022', 'Q4 2021', 'Q3 2021', 'Q2 2021', 'Q1 2021', 'Q4 2020', 'Q3 2020']

# Create DataFrame
df = pd.DataFrame(data, columns=columns)

# Print 
print(df)


In [ ]:
import matplotlib.pyplot as plt

# Scatter plot
plt.figure(figsize=(14, 8))
plt.scatter(df['County'], df['Q4 2023'], color='blue', label='Q4 2023')
plt.scatter(df['County'], df['Q3 2023'], color='red', label='Q3 2023')
plt.scatter(df['County'], df['Q2 2023'], color='green', label='Q2 2023')
plt.scatter(df['County'], df['Q1 2023'], color='orange', label='Q1 2023')
plt.scatter(df['County'], df['Q4 2022'], color='purple', label='Q4 2022')
plt.scatter(df['County'], df['Q3 2022'], color='cyan', label='Q3 2022')
plt.scatter(df['County'], df['Q2 2022'], color='magenta', label='Q2 2022')
plt.scatter(df['County'], df['Q1 2022'], color='yellow', label='Q1 2022')
plt.scatter(df['County'], df['Q4 2021'], color='black', label='Q4 2021')
plt.scatter(df['County'], df['Q3 2021'], color='brown', label='Q3 2021')
plt.scatter(df['County'], df['Q2 2021'], color='pink', label='Q2 2021')
plt.scatter(df['County'], df['Q1 2021'], color='gray', label='Q1 2021')
plt.scatter(df['County'], df['Q4 2020'], color='olive', label='Q4 2020')
plt.scatter(df['County'], df['Q3 2020'], color='cyan', label='Q3 2020')

plt.xlabel('County')
plt.ylabel('EV Charging Stations')
plt.title('EV Charging Station Adaptation by Year')
plt.xticks(rotation=90)
plt.legend()
plt.tight_layout()
plt.show()

# Bar plot
plt.figure(figsize=(14, 8))
plt.bar(df['County'], df['Q4 2023'], color='blue', label='Q4 2023')
plt.bar(df['County'], df['Q3 2023'], color='red', label='Q3 2023', alpha=0.7)
plt.bar(df['County'], df['Q2 2023'], color='green', label='Q2 2023', alpha=0.5)
plt.bar(df['County'], df['Q1 2023'], color='orange', label='Q1 2023', alpha=0.5)
plt.bar(df['County'], df['Q4 2022'], color='purple', label='Q4 2022', alpha=0.5)
plt.bar(df['County'], df['Q3 2022'], color='cyan', label='Q3 2022', alpha=0.5)
plt.bar(df['County'], df['Q2 2022'], color='magenta', label='Q2 2022', alpha=0.5)
plt.bar(df['County'], df['Q1 2022'], color='yellow', label='Q1 2022', alpha=0.5)
plt.bar(df['County'], df['Q4 2021'], color='black', label='Q4 2021', alpha=0.5)
plt.bar(df['County'], df['Q3 2021'], color='brown', label='Q3 2021', alpha=0.5)
plt.bar(df['County'], df['Q2 2021'], color='pink', label='Q2 2021', alpha=0.5)
plt.bar(df['County'], df['Q1 2021'], color='gray', label='Q1 2021', alpha=0.5)
plt.bar(df['County'], df['Q4 2020'], color='olive', label='Q4 2020', alpha=0.5)
plt.bar(df['County'], df['Q3 2020'], color='cyan', label='Q3 2020', alpha=0.5)

plt.xlabel('County')
plt.ylabel('EV Charging Stations')
plt.title('EV Charging Station Adaptation by Year')
plt.xticks(rotation=90)
plt.legend()
plt.tight_layout()
plt.show()

# Ending of Aram (EV_Chargers.ipynb)

# Beginning of Aram (Charging_Stations_by_County.ipynb)

In [ ]:
# Dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

#  Data
EV_Chargers_Path = "EV_Chargers1.csv"

# Read the data
EV_Chargers_data = pd.read_csv(EV_Chargers_Path)

# Preview data
EV_Chargers_data.head()

In [ ]:
# Data in the form of a list of lists
data = [
    ['Alameda', 5893, 4624, 4504, 4430, 4282, 5139, 5051, 3697, 3625, 3560, 3544, 3353, 3082, 2990],
    ['Alpine', 12, 10, 10, 6, 16, 16, 18, 14, 15, 15, 15, 15, 13, 12],
    ['Amador', 34, 34, 34, 34, 33, 33, 33, 39, 40, 40, 40, 40, 30, 29],
    ['Butte', 157, 149, 147, 146, 139, 158, 132, 94, 92, 91, 82, 76, 76, 74],
    ['Calaveras', 23, 23, 21, 25, 23, 23, 21, 21, 21, 19, 19, 7, 7, 7],
    ['Colusa', 37, 36, 36, 34, 34, 36, 36, 38, 34, 30, 28, 28, 28, 28],
    ['Contra Costa', 1632, 1734, 1674, 1503, 1495, 1699, 1715, 1451, 1483, 1461, 1455, 1355, 1325, 1068],
    ['Del Norte', 46, 47, 47, 39, 39, 32, 32, 29, 29, 25, 25, 30, 30, 30],
    ['El Dorado', 258, 214, 202, 189, 194, 193, 173, 179, 181, 177, 167, 173, 160, 150],
    ['Fresno', 1292, 1313, 1373, 1291, 1158, 1321, 1311, 1133, 1143, 1127, 1092, 921, 779, 684],
    ['Glenn', 8, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14],
    ['Humboldt', 157, 157, 157, 154, 150, 148, 145, 150, 142, 134, 124, 112, 114, 105],
    ['Imperial', 52, 50, 50, 49, 45, 43, 35, 31, 33, 33, 33, 27, 27, 15],
    ['Inyo', 83, 54, 52, 48, 49, 49, 41, 42, 39, 39, 35, 37, 23, 23],
    ['Kern', 832, 797, 721, 691, 646, 629, 631, 416, 359, 306, 298, 279, 279, 241],
    ['Kings', 431, 419, 410, 410, 176, 168, 160, 204, 148, 147, 147, 144, 140, 122],
    ['Lake', 18, 17, 14, 14, 15, 15, 14, 14, 17, 17, 15, 15, 15, 10],
    ['Lassen', 16, 16, 16, 16, 16, 16, 13, 13, 14, 6, 6, 5, 5, 4],
    ['Los Angeles', 31569, 29433, 29051, 27015, 25685, 24615, 23903, 20950, 20520, 20193, 19872, 19236, 18509, 16770],
    ['Madera', 203, 202, 203, 190, 161, 152, 147, 130, 122, 121, 121, 113, 89, 84],
    ['Marin', 1020, 1082, 1048, 1027, 933, 1052, 1037, 739, 734, 730, 728, 706, 684, 629],
    ['Mariposa', 62, 59, 60, 46, 33, 45, 35, 24, 30, 16, 16, 16, 16, 16],
    ['Mendocino', 229, 214, 203, 186, 200, 162, 158, 160, 158, 158, 146, 144, 129, 110],
    ['Merced', 182, 155, 124, 135, 177, 134, 139, 125, 142, 142, 138, 130, 114, 94],
    ['Modoc', 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 4, 4, 4, 4],
    ['Mono', 52, 38, 38, 38, 38, 37, 37, 32, 32, 32, 32, 32, 31, 31],
    ['Monterey', 491, 480, 487, 424, 394, 400, 383, 452, 425, 431, 424, 422, 360, 348],
    ['Napa', 478, 442, 433, 396, 391, 403, 393, 393, 390, 388, 386, 374, 348, 316],
    ['Nevada', 149, 144, 144, 169, 161, 152, 152, 116, 89, 88, 88, 93, 88, 86],
    ['Orange', 6771, 6002, 5797, 5383, 5214, 5189, 5003, 5852, 5657, 5477, 5381, 5242, 4886, 4799],
    ['Placer', 622, 522, 518, 455, 425, 448, 424, 419, 402, 365, 361, 351, 339, 305],
    ['Plumas', 7, 7, 7, 7, 7, 8, 7, 7, 7, 7, 7, 7, 5, 5],
    ['Riverside', 1835, 1835, 1745, 1635, 1508, 1493, 1441, 1548, 1495, 1441, 1422, 1464, 1360, 1306],
    ['Sacramento', 2253, 2205, 2149, 2040, 1756, 1839, 1765, 1813, 1836, 1852, 1850, 1777, 1765, 1438],
    ['San Benito', 27, 29, 28, 45, 43, 47, 47, 47, 49, 47, 47, 50, 42, 44],
    ['San Bernardino', 1957, 1841, 1767, 1682, 1615, 1560, 1424, 1367, 1478, 1415, 1395, 1338, 1297, 999],
    ['San Diego', 8583, 8094, 7960, 7735, 7358, 7074, 6551, 10080, 7278, 7199, 7170, 6874, 6760, 6073],
    ['San Francisco', 2431, 2070, 2002, 1952, 1436, 1575, 1519, 1630, 1840, 1671, 1609, 1505, 1384, 1264],
    ['San Joaquin', 825, 728, 680, 632, 612, 596, 565, 442, 407, 402, 397, 389, 326, 306],
    ['San Luis Obispo', 777, 705, 694, 650, 616, 640, 638, 566, 513, 513, 493, 493, 486, 442],
    ['San Mateo', 5533, 4798, 4619, 4557, 4436, 4799, 4688, 4511, 4701, 4459, 4282, 4070, 3758, 3617],
    ['Santa Barbara', 648, 526, 566, 512, 504, 515, 480, 557, 522, 492, 499, 494, 461, 437],
    ['Santa Clara', 20220, 18193, 17865, 17621, 16227, 16981, 16831, 15987, 16498, 16202, 16119, 15406, 14913, 14164],
    ['Santa Cruz', 359, 369, 356, 359, 345, 356, 333, 307, 306, 300, 302, 281, 262, 250],
    ['Shasta', 227, 179, 174, 153, 138, 105, 90, 86, 92, 93, 93, 96, 94, 57],
    ['Sierra', 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    ['Siskiyou', 77, 69, 68, 67, 67, 68, 68, 72, 68, 67, 56, 63, 63, 55],
    ['Solano', 608, 578, 560, 482, 473, 509, 501, 471, 421, 421, 394, 372, 362, 369],
    ['Sonoma', 902, 871, 845, 816, 794, 840, 765, 794, 803, 791, 789, 743, 695, 670],
    ['Stanislaus', 249, 202, 178, 188, 171, 158, 149, 151, 158, 150, 151, 145, 145, 122],
    ['Sutter', 35, 31, 24, 22, 27, 24, 24, 27, 33, 31, 31, 31, 23, 20],
    ['Tehama', 94, 91, 91, 87, 91, 80, 66, 35, 31, 30, 31, 32, 32, 32],
    ['Trinity', 15, 12, 14, 14, 14, 14, 14, 6, 6, 6, 6, 8, 8, 4],
    ['Tulare', 291, 275, 279, 231, 212, 160, 154, 123, 124, 119, 108, 103, 85, 76],
    ['Tuolumne', 55, 54, 54, 58, 60, 52, 52, 78, 54, 50, 50, 47, 47, 45],
    ['Ventura', 930, 875, 870, 858, 810, 821, 273, 14, 11, 11, 11, 11, 13, 749],
    ['Yolo', 468, 432, 427, 412, 398, 428, 679, 894, 894, 893, 892, 845, 868, 264],
    ['Yuba', 69, 80, 79, 75, 59, 60, 299, 394, 388, 386, 386, 324, 328, 14],
]

# Columns for the DataFrame
columns = ['County', 'Q4 2023', 'Q3 2023', 'Q2 2023', 'Q1 2023', 'Q4 2022', 'Q3 2022', 'Q2 2022', 
           'Q1 2022', 'Q4 2021', 'Q3 2021', 'Q2 2021', 'Q1 2021', 'Q4 2020', 'Q3 2020']

# Create DataFrame
df = pd.DataFrame(data, columns=columns)

# Print 
print(df)

In [ ]:
import matplotlib.pyplot as plt

# Scatter plot
plt.figure(figsize=(14, 8))
plt.scatter(df['County'], df['Q4 2023'], color='blue', label='Q4 2023')
plt.scatter(df['County'], df['Q3 2023'], color='red', label='Q3 2023')
plt.scatter(df['County'], df['Q2 2023'], color='green', label='Q2 2023')
plt.scatter(df['County'], df['Q1 2023'], color='orange', label='Q1 2023')
plt.scatter(df['County'], df['Q4 2022'], color='purple', label='Q4 2022')
plt.scatter(df['County'], df['Q3 2022'], color='cyan', label='Q3 2022')
plt.scatter(df['County'], df['Q2 2022'], color='magenta', label='Q2 2022')
plt.scatter(df['County'], df['Q1 2022'], color='yellow', label='Q1 2022')
plt.scatter(df['County'], df['Q4 2021'], color='black', label='Q4 2021')
plt.scatter(df['County'], df['Q3 2021'], color='brown', label='Q3 2021')
plt.scatter(df['County'], df['Q2 2021'], color='pink', label='Q2 2021')
plt.scatter(df['County'], df['Q1 2021'], color='gray', label='Q1 2021')
plt.scatter(df['County'], df['Q4 2020'], color='olive', label='Q4 2020')
plt.scatter(df['County'], df['Q3 2020'], color='cyan', label='Q3 2020')

plt.xlabel('County')
plt.ylabel('EV Charging Stations')
plt.title('EV Charging Station Adaptation by Year')
plt.xticks(rotation=90)
plt.legend()
plt.tight_layout()
plt.show()

# Bar plot
plt.figure(figsize=(14, 8))
plt.bar(df['County'], df['Q4 2023'], color='blue', label='Q4 2023')
plt.bar(df['County'], df['Q3 2023'], color='red', label='Q3 2023', alpha=0.7)
plt.bar(df['County'], df['Q2 2023'], color='green', label='Q2 2023', alpha=0.5)
plt.bar(df['County'], df['Q1 2023'], color='orange', label='Q1 2023', alpha=0.5)
plt.bar(df['County'], df['Q4 2022'], color='purple', label='Q4 2022', alpha=0.5)
plt.bar(df['County'], df['Q3 2022'], color='cyan', label='Q3 2022', alpha=0.5)
plt.bar(df['County'], df['Q2 2022'], color='magenta', label='Q2 2022', alpha=0.5)
plt.bar(df['County'], df['Q1 2022'], color='yellow', label='Q1 2022', alpha=0.5)
plt.bar(df['County'], df['Q4 2021'], color='black', label='Q4 2021', alpha=0.5)
plt.bar(df['County'], df['Q3 2021'], color='brown', label='Q3 2021', alpha=0.5)
plt.bar(df['County'], df['Q2 2021'], color='pink', label='Q2 2021', alpha=0.5)
plt.bar(df['County'], df['Q1 2021'], color='gray', label='Q1 2021', alpha=0.5)
plt.bar(df['County'], df['Q4 2020'], color='olive', label='Q4 2020', alpha=0.5)
plt.bar(df['County'], df['Q3 2020'], color='cyan', label='Q3 2020', alpha=0.5)

plt.xlabel('County')
plt.ylabel('EV Charging Stations')
plt.title('EV Charging Station Adaptation by Year')
plt.xticks(rotation=90)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Selecting a subset of counties for demonstration
counties_to_plot = ['Los Angeles', 'San Francisco', 'Orange', 'Alameda', 'San Diego', 'Santa Clara', 'San Mateo']

plt.figure(figsize=(10, 6))

for county in counties_to_plot:
    county_data = df[df['County'] == county].iloc[:, 1:]  # Exclude 'County' column
    county_data = county_data.T.reset_index()
    county_data.columns = ['Quarter', 'EV Charging Stations']
    
    plt.plot(county_data['Quarter'], county_data['EV Charging Stations'], marker='o', label=county)

plt.xlabel('Quarter')
plt.ylabel('EV Charging Stations')
plt.title('EV Charging Station Trend by County')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Beginning of Mo (gas_price.ipynb)

In [ ]:
import requests
import json


In [ ]:
api_key = '4k9wboAV8W8ZdRVGqA9uIR:4LYhjC1Sx8VVuzdgg7P2uu'

In [ ]:
from pprint import pprint

In [ ]:
# url = "https://collectapi.com/api/gasPrice/gas-prices-api?tab=pricing"
url = "https://api.collectapi.com/gasPrice/stateUsaPrice?state="

In [ ]:
headers = {
    'content-type': "application/json",
    'authorization': "apikey 3NHVADT5KuNLk3ZMn0THtC:2pmqCh12nXGVUWBbdn6wFy"
}

In [ ]:
headers

In [ ]:
response = requests.get(url + "CA", headers=headers)

In [ ]:
response.status_code

In [ ]:
import http.client

conn = http.client.HTTPSConnection("api.collectapi.com")

headers = {
    'content-type': "application/json",
    'authorization': "apikey 3NHVADT5KuNLk3ZMn0THtC:2pmqCh12nXGVUWBbdn6wFy"
    }

conn.request("GET", "/gasPrice/stateUsaPrice?state=WA", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

In [ ]:
pprint(response.json())

In [ ]:
# api_key = "&apikey=" + api_key

In [ ]:
import pandas as pd

In [ ]:
data = response.json()

In [ ]:
data = [{'currency': 'usd', 'gasoline': '4.834', 'midGrade': '5.059', 'premium': '5.246', 'diesel': '5.620', 'name': 'Bakersfield', 'lowerName': 'bakersfield'}, {'currency': 'usd', 'gasoline': '4.817', 'midGrade': '5.035', 'premium': '5.156', 'diesel': '5.130', 'name': 'Chico-Paradise', 'lowerName': 'chico-paradise'}, {'currency': 'usd', 'gasoline': '4.838', 'midGrade': '5.040', 'premium': '5.216', 'diesel': '5.393', 'name': 'Fresno', 'lowerName': 'fresno'}, {'currency': 'usd', 'gasoline': '4.975', 'midGrade': '5.187', 'premium': '5.330', 'diesel': '5.481', 'name': 'Los Angeles-Long Beach', 'lowerName': 'los angeles-long beach'}, {'currency': 'usd', 'gasoline': '4.894', 'midGrade': '5.099', 'premium': '5.279', 'diesel': '5.311', 'name': 'Merced', 'lowerName': 'merced'}, {'currency': 'usd', 'gasoline': '4.771', 'midGrade': '5.011', 'premium': '5.180', 'diesel': '5.300', 'name': 'Modesto', 'lowerName': 'modesto'}, {'currency': 'usd', 'gasoline': '4.993', 'midGrade': '5.206', 'premium': '5.387', 'diesel': '5.429', 'name': 'Oakland', 'lowerName': 'oakland'}, {'currency': 'usd', 'gasoline': '4.933', 'midGrade': '5.135', 'premium': '5.267', 'diesel': '5.278', 'name': 'Orange County', 'lowerName': 'orange county'}, {'currency': 'usd', 'gasoline': '4.765', 'midGrade': '4.918', 'premium': '5.064', 'diesel': '5.094', 'name': 'Redding', 'lowerName': 'redding'}, {'currency': 'usd', 'gasoline': '4.853', 'midGrade': '5.070', 'premium': '5.220', 'diesel': '5.282', 'name': 'Riverside', 'lowerName': 'riverside'}, {'currency': 'usd', 'gasoline': '4.940', 'midGrade': '5.141', 'premium': '5.324', 'diesel': '5.258', 'name': 'Sacramento', 'lowerName': 'sacramento'}, {'currency': 'usd', 'gasoline': '5.061', 'midGrade': '5.261', 'premium': '5.450', 'diesel': '5.621', 'name': 'Salinas', 'lowerName': 'salinas'}, {'currency': 'usd', 'gasoline': '4.972', 'midGrade': '5.180', 'premium': '5.335', 'diesel': '5.328', 'name': 'San Diego', 'lowerName': 'san diego'}, {'currency': 'usd', 'gasoline': '5.111', 'midGrade': '5.307', 'premium': '5.500', 'diesel': '5.529', 'name': 'San Francisco', 'lowerName': 'san francisco'}, {'currency': 'usd', 'gasoline': '4.952', 'midGrade': '5.154', 'premium': '5.327', 'diesel': '5.383', 'name': 'San Jose', 'lowerName': 'san jose'}, {'currency': 'usd', 'gasoline': '5.080', 'midGrade': '5.284', 'premium': '5.449', 'diesel': '5.808', 'name': 'San Luis Obispo-Atascadero-Paso Robles', 'lowerName': 'san luis obispo-atascadero-paso robles'}, {'currency': 'usd', 'gasoline': '4.971', 'midGrade': '5.162', 'premium': '5.304', 'diesel': '5.517', 'name': 'Santa Barbara-Santa Maria-Lompoc', 'lowerName': 'santa barbara-santa maria-lompoc'}, {'currency': 'usd', 'gasoline': '4.969', 'midGrade': '5.187', 'premium': '5.345', 'diesel': '5.556', 'name': 'Santa Cruz-Watsonville', 'lowerName': 'santa cruz-watsonville'}, {'currency': 'usd', 'gasoline': '5.098', 'midGrade': '5.301', 'premium': '5.472', 'diesel': '5.458', 'name': 'Santa Rosa', 'lowerName': 'santa rosa'}, {'currency': 'usd', 'gasoline': '4.828', 'midGrade': '5.046', 'premium': '5.226', 'diesel': '5.344', 'name': 'Stockton-Lodi', 'lowerName': 'stockton-lodi'}, {'currency': 'usd', 'gasoline': '4.907', 'midGrade': '5.112', 'premium': '5.280', 'diesel': '5.298', 'name': 'Vallejo-Fairfield', 'lowerName': 'vallejo-fairfield'}, {'currency': 'usd', 'gasoline': '5.015', 'midGrade': '5.218', 'premium': '5.362', 'diesel': '5.557', 'name': 'Ventura', 'lowerName': 'ventura'}, {'currency': 'usd', 'gasoline': '4.844', 'midGrade': '5.026', 'premium': '5.240', 'diesel': '5.458', 'name': 'Visalia-Tulare-Porterville', 'lowerName': 'visalia-tulare-porterville'}, {'currency': 'usd', 'gasoline': '4.900', 'midGrade': '5.132', 'premium': '5.343', 'diesel': '5.154', 'name': 'Yolo', 'lowerName': 'yolo'}, {'currency': 'usd', 'gasoline': '4.744', 'midGrade': '4.936', 'premium': '5.135', 'diesel': '5.158', 'name': 'Yuba City', 'lowerName': 'yuba city'}, {'currency': 'usd', 'gasoline': '4.772', 'midGrade': '4.962', 'premium': '5.187', 'diesel': '5.526', 'name': 'El Centro', 'lowerName': 'el centro'}, {'currency': 'usd', 'gasoline': '4.745', 'midGrade': '4.931', 'premium': '5.123', 'diesel': '5.316', 'name': 'Hanford-Corcoran', 'lowerName': 'hanford-corcoran'}, {'currency': 'usd', 'gasoline': '4.811', 'midGrade': '4.986', 'premium': '5.268', 'diesel': '5.532', 'name': 'Madera-Chowchilla', 'lowerName': 'madera-chowchilla'}, {'currency': 'usd', 'gasoline': '5.081', 'midGrade': '5.273', 'premium': '5.453', 'diesel': '5.391', 'name': 'Napa', 'lowerName': 'napa'}, {'currency': 'usd', 'gasoline': '4.891', 'midGrade': '5.078', 'premium': '5.243', 'diesel': '5.320', 'name': 'San Bernardino', 'lowerName': 'san bernardino'}, {'currency': 'usd', 'gasoline': '5.077', 'midGrade': '5.249', 'premium': '5.464', 'diesel': '5.460', 'name': 'San Rafael', 'lowerName': 'san rafael'}]


In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.columns = ["currency", "gasoline", "midGrade", "premium", "diesel", "name", "lowerName"]

In [ ]:
df.to_excel('gas_price_3.xlsx',index=False)

# Ending of Mo

# Beginning of Jim (presentation_1_jlh.ipynb)